In [1]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, BatchNormalization
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import optimizers

import pickle
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
datagen = ImageDataGenerator(rescale = 1/255, rotation_range=30, width_shift_range=0.1, height_shift_range=0.1,
                             horizontal_flip=True, vertical_flip=True)

In [3]:
train_data_dir = "dataset/CASIA2/train"
valid_data_dir = "dataset/CASIA2/valid"
img_width = 348
img_height = 256
batch_size = 32



In [4]:
train_gen = datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode="binary",
    shuffle=True)

step_size_train = train_gen.n // train_gen.batch_size

Found 11353 images belonging to 2 classes.


In [5]:
valid_gen = datagen.flow_from_directory(
    valid_data_dir,
    target_size=(img_width, img_height),
    batch_size=batch_size,
    class_mode="binary",
    shuffle=True)

step_size_valid = valid_gen.n // valid_gen.batch_size

Found 1261 images belonging to 2 classes.


In [7]:
cnn_model=Sequential()

cnn_model.add(Conv2D(filters=16, kernel_size=(32,32), strides=(14, 16), padding='valid', activation='relu', input_shape= (348,256,3)))

cnn_model.add(Dropout(0.2))
cnn_model.add(BatchNormalization())


cnn_model.add(Conv2D(filters=32, kernel_size=(20,12), strides=4, padding='valid', activation='relu'))

cnn_model.add(Dropout(0.2))
cnn_model.add(BatchNormalization())

cnn_model.add(Flatten())

cnn_model.add(Dense(64, activation='relu'))

cnn_model.add(Dropout(0.2))
cnn_model.add(BatchNormalization())

cnn_model.add(Dense(1, activation='sigmoid'))

adam = optimizers.Adam(lr=0.1, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)

cnn_model.compile(optimizer="adagrad", loss='binary_crossentropy', metrics=['accuracy'])

epochs = 100

cnn_model.fit_generator(generator=train_gen,
                    steps_per_epoch=step_size_train,
                    validation_data=valid_gen,
                    validation_steps=step_size_valid,
                    epochs=epochs
)


Epoch 1/100
355/355 [==============================] - 262s 738ms/step - loss: 0.7086 - acc: 0.5741 - val_loss: 0.6737 - val_acc: 0.5924
Epoch 2/100
355/355 [==============================] - 260s 733ms/step - loss: 0.6790 - acc: 0.5897 - val_loss: 0.6696 - val_acc: 0.6075
Epoch 3/100
355/355 [==============================] - 282s 796ms/step - loss: 0.6727 - acc: 0.6035 - val_loss: 0.6644 - val_acc: 0.6217
Epoch 4/100
355/355 [==============================] - 303s 853ms/step - loss: 0.6700 - acc: 0.5992 - val_loss: 0.6631 - val_acc: 0.6106
Epoch 5/100
355/355 [==============================] - 255s 717ms/step - loss: 0.6660 - acc: 0.5989 - val_loss: 0.6551 - val_acc: 0.6178
Epoch 6/100
355/355 [==============================] - 256s 720ms/step - loss: 0.6648 - acc: 0.6018 - val_loss: 0.6585 - val_acc: 0.6241
Epoch 7/100
355/355 [==============================] - 255s 719ms/step - loss: 0.6611 - acc: 0.6088 - val_loss: 0.6572 - val_acc: 0.6193
Epoch 8/100
355/355 [====================

Epoch 38/100
355/355 [==============================] - 256s 722ms/step - loss: 0.6425 - acc: 0.6288 - val_loss: 0.6445 - val_acc: 0.6400
Epoch 39/100
355/355 [==============================] - 255s 719ms/step - loss: 0.6435 - acc: 0.6281 - val_loss: 0.6566 - val_acc: 0.6193
Epoch 40/100
355/355 [==============================] - 255s 718ms/step - loss: 0.6395 - acc: 0.6372 - val_loss: 0.6490 - val_acc: 0.6352
Epoch 41/100
355/355 [==============================] - 254s 715ms/step - loss: 0.6426 - acc: 0.6280 - val_loss: 0.6535 - val_acc: 0.6178
Epoch 42/100
355/355 [==============================] - 255s 720ms/step - loss: 0.6441 - acc: 0.6272 - val_loss: 0.6564 - val_acc: 0.6217
Epoch 43/100
355/355 [==============================] - 256s 720ms/step - loss: 0.6428 - acc: 0.6231 - val_loss: 0.6601 - val_acc: 0.6193
Epoch 44/100
355/355 [==============================] - 255s 719ms/step - loss: 0.6437 - acc: 0.6306 - val_loss: 0.6492 - val_acc: 0.6273
Epoch 45/100
355/355 [============

Epoch 75/100
355/355 [==============================] - 257s 723ms/step - loss: 0.6380 - acc: 0.6312 - val_loss: 0.6528 - val_acc: 0.6312
Epoch 76/100
355/355 [==============================] - 255s 718ms/step - loss: 0.6384 - acc: 0.6287 - val_loss: 0.6519 - val_acc: 0.6273
Epoch 77/100
355/355 [==============================] - 255s 719ms/step - loss: 0.6344 - acc: 0.6389 - val_loss: 0.6398 - val_acc: 0.6416
Epoch 78/100
355/355 [==============================] - 256s 720ms/step - loss: 0.6357 - acc: 0.6369 - val_loss: 0.6447 - val_acc: 0.6257
Epoch 79/100
355/355 [==============================] - 253s 712ms/step - loss: 0.6353 - acc: 0.6363 - val_loss: 0.6423 - val_acc: 0.6368
Epoch 80/100
355/355 [==============================] - 256s 720ms/step - loss: 0.6367 - acc: 0.6346 - val_loss: 0.6464 - val_acc: 0.6289
Epoch 81/100
355/355 [==============================] - 255s 719ms/step - loss: 0.6336 - acc: 0.6353 - val_loss: 0.6401 - val_acc: 0.6455
Epoch 82/100
355/355 [============

In [8]:
cnn_model.save("Trained_Model.model")